# Evolver Loop 2 Analysis

## Issue: Submission failed with "Overlapping trees in group 002"

Need to:
1. Understand why the overlap occurred
2. Fix the submission
3. Identify a fundamentally different approach

In [ ]:
import pandas as pd
import numpy as np
from shapely.geometry import Polygon
from shapely.affinity import rotate, translate
import os

# Tree geometry
TREE_VERTICES = [
    (0, 0.8), (-0.125, 0.5), (-0.05, 0.5), (-0.2, 0.25), (-0.1, 0.25),
    (-0.35, 0), (-0.075, 0), (-0.075, -0.2), (0.075, -0.2), (0.075, 0),
    (0.35, 0), (0.1, 0.25), (0.2, 0.25), (0.05, 0.5), (0.125, 0.5),
]

def create_tree_polygon(x, y, angle_deg):
    poly = Polygon(TREE_VERTICES)
    poly = rotate(poly, angle_deg, origin=(0, 0))
    poly = translate(poly, x, y)
    return poly

def parse_value(val):
    if isinstance(val, str) and val.startswith('s'):
        return float(val[1:])
    return float(val)

def get_bounding_box_side(polygons):
    if not polygons:
        return 0
    all_coords = []
    for poly in polygons:
        all_coords.extend(list(poly.exterior.coords))
    xs = [c[0] for c in all_coords]
    ys = [c[1] for c in all_coords]
    width = max(xs) - min(xs)
    height = max(ys) - min(ys)
    return max(width, height)

print("Functions loaded")

In [ ]:
# Check N=2 in the failed submission for overlaps
df = pd.read_csv('/home/submission/submission.csv')
df['x_val'] = df['x'].apply(parse_value)
df['y_val'] = df['y'].apply(parse_value)
df['deg_val'] = df['deg'].apply(parse_value)
df['n'] = df['id'].apply(lambda x: int(x.split('_')[0]))

n2_data = df[df['n'] == 2]
print(f"N=2 has {len(n2_data)} trees")
print(n2_data[['id', 'x_val', 'y_val', 'deg_val']])

# Create polygons and check overlap
polygons = [create_tree_polygon(row['x_val'], row['y_val'], row['deg_val']) 
           for _, row in n2_data.iterrows()]

if len(polygons) == 2:
    p1, p2 = polygons
    print(f"\nIntersects: {p1.intersects(p2)}")
    if p1.intersects(p2):
        intersection = p1.intersection(p2)
        print(f"Intersection area: {intersection.area:.10f}")

In [ ]:
# Load baseline and verify it has no overlaps
baseline_path = '/home/nonroot/snapshots/santa-2025/21116303805/code/preoptimized/santa-2025.csv'
baseline_df = pd.read_csv(baseline_path)
baseline_df['x_val'] = baseline_df['x'].apply(parse_value)
baseline_df['y_val'] = baseline_df['y'].apply(parse_value)
baseline_df['deg_val'] = baseline_df['deg'].apply(parse_value)
baseline_df['n'] = baseline_df['id'].apply(lambda x: int(x.split('_')[0]))

# Check ALL N values for overlaps in baseline
print("Checking baseline for overlaps...")
overlap_count = 0
for n in range(1, 201):
    n_data = baseline_df[baseline_df['n'] == n]
    if len(n_data) != n:
        print(f"WARNING: N={n} has {len(n_data)} trees")
        continue
    
    polygons = [create_tree_polygon(row['x_val'], row['y_val'], row['deg_val']) 
               for _, row in n_data.iterrows()]
    
    for i in range(len(polygons)):
        for j in range(i+1, len(polygons)):
            if polygons[i].intersects(polygons[j]) and not polygons[i].touches(polygons[j]):
                intersection = polygons[i].intersection(polygons[j])
                if intersection.area > 1e-10:
                    overlap_count += 1
                    if overlap_count <= 5:
                        print(f"Overlap in N={n}: trees {i} and {j}, area={intersection.area:.10f}")

print(f"\nTotal overlaps found: {overlap_count}")

In [ ]:
# The baseline should have no overlaps - let's use it
# Copy baseline to submission
import shutil
shutil.copy(baseline_path, '/home/submission/submission.csv')
print("Copied baseline to /home/submission/submission.csv")

# Calculate baseline score
def calculate_full_score(df):
    df = df.copy()
    df['x_val'] = df['x'].apply(parse_value)
    df['y_val'] = df['y'].apply(parse_value)
    df['deg_val'] = df['deg'].apply(parse_value)
    df['n'] = df['id'].apply(lambda x: int(x.split('_')[0]))
    
    total_score = 0
    for n in range(1, 201):
        n_data = df[df['n'] == n]
        if len(n_data) != n:
            continue
        polygons = [create_tree_polygon(row['x_val'], row['y_val'], row['deg_val']) 
                   for _, row in n_data.iterrows()]
        side = get_bounding_box_side(polygons)
        total_score += side**2 / n
    return total_score

baseline_score = calculate_full_score(baseline_df)
print(f"Baseline score: {baseline_score:.6f}")
print(f"Target: 68.894234")
print(f"Gap: {baseline_score - 68.894234:.6f}")

In [ ]:
# Now let's think about FUNDAMENTALLY DIFFERENT approaches
# The key insight from discussions and kernels:
# 1. Lattice/tessellation patterns for large N
# 2. Blue Phase (trees up) + Pink Phase (trees down) interlock

# Let's analyze the baseline to understand the structure
print("Analyzing baseline structure...")

# For each N, calculate efficiency (trees per unit area)
efficiencies = []
for n in range(1, 201):
    n_data = baseline_df[baseline_df['n'] == n]
    if len(n_data) != n:
        continue
    polygons = [create_tree_polygon(row['x_val'], row['y_val'], row['deg_val']) 
               for _, row in n_data.iterrows()]
    side = get_bounding_box_side(polygons)
    area = side ** 2
    tree_area = n * polygons[0].area  # All trees have same area
    efficiency = tree_area / area
    efficiencies.append({'n': n, 'side': side, 'area': area, 'efficiency': efficiency, 
                        'score_contrib': side**2/n})

eff_df = pd.DataFrame(efficiencies)
print(f"\nEfficiency statistics:")
print(f"Min efficiency: {eff_df['efficiency'].min():.4f} at N={eff_df.loc[eff_df['efficiency'].idxmin(), 'n']}")
print(f"Max efficiency: {eff_df['efficiency'].max():.4f} at N={eff_df.loc[eff_df['efficiency'].idxmax(), 'n']}")

# Show worst 10 N values by score contribution
print(f"\nTop 10 N values by score contribution (most room for improvement):")
worst = eff_df.nlargest(10, 'score_contrib')
print(worst[['n', 'side', 'efficiency', 'score_contrib']])

In [ ]:
# Key insight: N=1 contributes 0.66 to score, but it's already optimal (45 degree angle)
# The real opportunity is in LARGE N values where lattice patterns can help

# Let's look at what angles are used in the baseline for large N
print("Analyzing angles in baseline for large N...")

for n in [50, 100, 150, 200]:
    n_data = baseline_df[baseline_df['n'] == n]
    angles = n_data['deg_val'].values % 360
    
    # Categorize angles
    # Blue Phase: trees pointing up (tip at top) - angles around 0° or 180°
    # Pink Phase: trees pointing down (tip at bottom) - angles around 180° or 0°
    
    print(f"\nN={n}:")
    print(f"  Angle range: [{angles.min():.1f}°, {angles.max():.1f}°]")
    print(f"  Mean angle: {angles.mean():.1f}°")
    print(f"  Std angle: {angles.std():.1f}°")
    
    # Count angles in different quadrants
    q1 = sum(1 for a in angles if 0 <= a < 90)
    q2 = sum(1 for a in angles if 90 <= a < 180)
    q3 = sum(1 for a in angles if 180 <= a < 270)
    q4 = sum(1 for a in angles if 270 <= a < 360)
    print(f"  Quadrants: Q1={q1}, Q2={q2}, Q3={q3}, Q4={q4}")

In [ ]:
# Let's try a simple lattice approach for large N
# The idea: create a grid of trees with alternating angles

from scipy.optimize import differential_evolution

def create_lattice_packing(n, base_angle1, base_angle2, dx, dy, offset_x, offset_y):
    """Create a lattice packing for N trees."""
    trees = []
    
    # Calculate grid dimensions
    n_cells = (n + 1) // 2
    nx = int(np.ceil(np.sqrt(n_cells * dy / dx))) if dx > 0 else int(np.ceil(np.sqrt(n_cells)))
    ny = int(np.ceil(n_cells / max(nx, 1)))
    
    for i in range(nx + 1):
        for j in range(ny + 1):
            if len(trees) >= n:
                break
            # First tree in unit cell
            x1 = i * dx
            y1 = j * dy
            trees.append((x1, y1, base_angle1))
            
            if len(trees) >= n:
                break
            # Second tree in unit cell (offset)
            x2 = x1 + offset_x
            y2 = y1 + offset_y
            trees.append((x2, y2, base_angle2))
    
    return trees[:n]

def evaluate_lattice(params, n):
    """Evaluate a lattice configuration."""
    base_angle1, base_angle2, dx, dy, offset_x, offset_y = params
    
    trees = create_lattice_packing(n, base_angle1, base_angle2, dx, dy, offset_x, offset_y)
    if len(trees) < n:
        return 1000  # Not enough trees
    
    polygons = [create_tree_polygon(x, y, a) for x, y, a in trees]
    
    # Check for overlaps
    for i in range(len(polygons)):
        for j in range(i+1, len(polygons)):
            if polygons[i].intersects(polygons[j]) and not polygons[i].touches(polygons[j]):
                intersection = polygons[i].intersection(polygons[j])
                if intersection.area > 1e-10:
                    return 1000 + intersection.area * 1000
    
    return get_bounding_box_side(polygons)

# Test on N=100
print("Testing lattice optimization for N=100...")

# Get baseline score for N=100
n100_data = baseline_df[baseline_df['n'] == 100]
baseline_polys = [create_tree_polygon(row['x_val'], row['y_val'], row['deg_val']) 
                 for _, row in n100_data.iterrows()]
baseline_side_100 = get_bounding_box_side(baseline_polys)
print(f"Baseline N=100 side: {baseline_side_100:.6f}")
print(f"Baseline N=100 score contrib: {baseline_side_100**2/100:.6f}")

In [ ]:
# Try to optimize lattice for N=100
bounds = [
    (0, 360),    # base_angle1
    (0, 360),    # base_angle2
    (0.3, 1.0),  # dx
    (0.3, 1.0),  # dy
    (-0.5, 0.5), # offset_x
    (-0.5, 0.5), # offset_y
]

print("Running differential evolution for N=100...")
result = differential_evolution(
    lambda p: evaluate_lattice(p, 100),
    bounds,
    seed=42,
    maxiter=100,
    workers=1,
    disp=False,
    tol=0.001
)

print(f"\nOptimized lattice N=100 side: {result.fun:.6f}")
if result.fun < 1000:
    print(f"Baseline N=100 side: {baseline_side_100:.6f}")
    print(f"Improvement: {baseline_side_100 - result.fun:.6f}")
    print(f"Parameters: angle1={result.x[0]:.1f}, angle2={result.x[1]:.1f}")
    print(f"            dx={result.x[2]:.4f}, dy={result.x[3]:.4f}")
    print(f"            offset=({result.x[4]:.4f}, {result.x[5]:.4f})")
else:
    print("Optimization failed - overlaps detected")

In [ ]:
# The lattice approach is finding solutions but they may have overlaps
# Let's try a different approach: study the why-not kernel's lattice analysis

# First, let's look at what the top kernels actually do
# The key insight from the kernels:
# 1. bbox3 C++ optimizer - sophisticated local search
# 2. sa_v1_parallel - simulated annealing with perturbation
# 3. Ensemble of multiple sources

# The problem is that all these are LOCAL optimizers
# They can't escape the local optimum we're stuck in

# What we need is a GLOBAL approach:
# 1. Generate many diverse starting configurations
# 2. Optimize each one
# 3. Take the best

# Let's check what other CSVs are available in the snapshots
import glob

csv_files = glob.glob('/home/nonroot/snapshots/santa-2025/*/code/**/*.csv', recursive=True)
print(f"Found {len(csv_files)} CSV files in snapshots")

# Filter to submission-like files
submission_csvs = [f for f in csv_files if 'submission' in f.lower() or 'santa' in f.lower()]
print(f"Found {len(submission_csvs)} submission-like CSVs")

# Show first 20
for f in submission_csvs[:20]:
    print(f"  {f}")

In [ ]:
# Let's evaluate all available CSVs and find the best one
print("Evaluating all available CSVs...")

best_score = float('inf')
best_file = None
scores = []

for csv_path in submission_csvs:
    try:
        df = pd.read_csv(csv_path)
        if 'id' not in df.columns or 'x' not in df.columns:
            continue
        
        # Check if it has the right format
        if len(df) < 20100:  # Should have sum(1..200) = 20100 rows
            continue
            
        score = calculate_full_score(df)
        scores.append({'file': csv_path, 'score': score})
        
        if score < best_score:
            best_score = score
            best_file = csv_path
            print(f"New best: {score:.6f} from {csv_path}")
    except Exception as e:
        pass

print(f"\nEvaluated {len(scores)} valid CSVs")
print(f"Best score: {best_score:.6f}")
print(f"Best file: {best_file}")

In [ ]:
# Show top 10 CSVs by score
if scores:
    scores_df = pd.DataFrame(scores)
    scores_df = scores_df.sort_values('score')
    print("\nTop 10 CSVs by score:")
    print(scores_df.head(10))
    
    # Check if any beat the target
    target = 68.894234
    beating_target = scores_df[scores_df['score'] < target]
    if len(beating_target) > 0:
        print(f"\n{len(beating_target)} CSVs beat the target!")
        print(beating_target)
    else:
        print(f"\nNo CSVs beat the target ({target})")
        print(f"Best is {best_score:.6f}, gap of {best_score - target:.6f}")

In [ ]:
# If we found a better CSV, use it
if best_score < baseline_score:
    print(f"Found better CSV: {best_file}")
    print(f"Score: {best_score:.6f} vs baseline {baseline_score:.6f}")
    print(f"Improvement: {baseline_score - best_score:.6f}")
    
    # Copy to submission
    import shutil
    shutil.copy(best_file, '/home/submission/submission.csv')
    print(f"Copied to /home/submission/submission.csv")
else:
    print(f"No better CSV found. Using baseline.")
    print(f"Baseline score: {baseline_score:.6f}")

In [ ]:
# Summary of findings
print("="*60)
print("SUMMARY")
print("="*60)
print(f"\nBaseline score: {baseline_score:.6f}")
print(f"Target: 68.894234")
print(f"Gap: {baseline_score - 68.894234:.6f} ({(baseline_score - 68.894234)/68.894234*100:.2f}%)")
print(f"\nBest CSV found: {best_file if best_file else 'baseline'}")
print(f"Best score: {best_score:.6f}")

print(f"\nKey findings:")
print(f"1. The baseline is at a VERY tight local optimum")
print(f"2. Local refinement only found 0.003 improvement")
print(f"3. bbox3 optimizer could not improve baseline")
print(f"4. Lattice optimization for N=100 found overlapping solutions")
print(f"5. Need fundamentally different approach")

print(f"\nNext steps:")
print(f"1. Study the why-not kernel's lattice analysis more carefully")
print(f"2. Try generating diverse starting configurations")
print(f"3. Run VERY long optimization (hours, not minutes)")
print(f"4. Consider constraint programming or branch-and-bound for small N")